Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [24]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [25]:
loc = './data/'
pickle_file = loc+'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [26]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [27]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [37]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print ("initial",hidden.get_shape().as_list(), ":", conv.get_shape().as_list(), "::", layer1_biases.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print ("second",hidden.get_shape().as_list() , ":", conv.get_shape().as_list(), "::", layer2_biases.get_shape().as_list())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print ("reshape", reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

initial [16, 14, 14, 16] : [16, 14, 14, 16] :: [16]
second [16, 7, 7, 16] : [16, 7, 7, 16] :: [16]
reshape [16, 784]
initial [10000, 14, 14, 16] : [10000, 14, 14, 16] :: [16]
second [10000, 7, 7, 16] : [10000, 7, 7, 16] :: [16]
reshape [10000, 784]
initial [10000, 14, 14, 16] : [10000, 14, 14, 16] :: [16]
second [10000, 7, 7, 16] : [10000, 7, 7, 16] :: [16]
reshape [10000, 784]


In [30]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.193368
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.019018
Minibatch accuracy: 18.8%
Validation accuracy: 36.4%
Minibatch loss at step 100: 1.450402
Minibatch accuracy: 68.8%
Validation accuracy: 62.7%
Minibatch loss at step 150: 0.783025
Minibatch accuracy: 62.5%
Validation accuracy: 73.6%
Minibatch loss at step 200: 0.804161
Minibatch accuracy: 62.5%
Validation accuracy: 76.1%
Minibatch loss at step 250: 0.969772
Minibatch accuracy: 62.5%
Validation accuracy: 71.2%
Minibatch loss at step 300: 0.524829
Minibatch accuracy: 87.5%
Validation accuracy: 77.3%
Minibatch loss at step 350: 0.767558
Minibatch accuracy: 68.8%
Validation accuracy: 77.0%
Minibatch loss at step 400: 0.638982
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.612974
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.633526
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [65]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) 
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print ("initial",hidden.get_shape().as_list(), ":", conv.get_shape().as_list(), "::", layer1_biases.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print ("second",hidden.get_shape().as_list() , ":", conv.get_shape().as_list(), "::", layer2_biases.get_shape().as_list())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    x = tf.nn.relu(tf.matmul(reshape, layer3_weights))
    print("x", x.get_shape().as_list())
    print("l3b",layer3_biases.get_shape().as_list() )
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

initial [16, 14, 14, 16] : [16, 28, 28, 16] :: [16]
second [16, 7, 7, 16] : [16, 14, 14, 16] :: [16]
x [16, 64]
l3b [64]
initial [10000, 14, 14, 16] : [10000, 28, 28, 16] :: [16]
second [10000, 7, 7, 16] : [10000, 14, 14, 16] :: [16]
x [10000, 64]
l3b [64]
initial [10000, 14, 14, 16] : [10000, 28, 28, 16] :: [16]
second [10000, 7, 7, 16] : [10000, 14, 14, 16] :: [16]
x [10000, 64]
l3b [64]


In [66]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.944936
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.989896
Minibatch accuracy: 31.2%
Validation accuracy: 29.2%
Minibatch loss at step 100: 1.443438
Minibatch accuracy: 62.5%
Validation accuracy: 54.9%
Minibatch loss at step 150: 0.687074
Minibatch accuracy: 68.8%
Validation accuracy: 73.5%
Minibatch loss at step 200: 0.637908
Minibatch accuracy: 68.8%
Validation accuracy: 74.9%
Minibatch loss at step 250: 1.118171
Minibatch accuracy: 50.0%
Validation accuracy: 77.8%
Minibatch loss at step 300: 0.550538
Minibatch accuracy: 93.8%
Validation accuracy: 77.0%
Minibatch loss at step 350: 0.776087
Minibatch accuracy: 68.8%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.721791
Minibatch accuracy: 75.0%
Validation accuracy: 81.1%
Minibatch loss at step 450: 0.793039
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 500: 0.550355
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [103]:
batch_size = 16
patch_size = 5
depth = 32
num_hidden = 64
droupout_keep_prob = 0.75

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
#  layer2_5_weights = tf.Variable(tf.truncated_normal(
#      [patch_size, patch_size, depth, depth], stddev=0.1))
#  layer2_5_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden_conv = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden_conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    #hidden = tf.nn.dropout(hidden_pool, droupout_keep_prob) 
    print ("initial",hidden.get_shape().as_list(), ":", conv.get_shape().as_list(), "::", layer1_biases.get_shape().as_list())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    print ("second",hidden.get_shape().as_list(), ":", conv.get_shape().as_list(), "::", layer1_biases.get_shape().as_list())
    hidden_conv = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(hidden_conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    ##
    #conv = tf.nn.conv2d(hidden, layer2_5_weights, [1, 1, 1, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer2_5_biases)

    #hidden = tf.nn.dropout(hidden_pool, droupout_keep_prob) 
    print ("third",hidden.get_shape().as_list() , ":", conv.get_shape().as_list(), "::", layer2_biases.get_shape().as_list())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.05
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                          100, 0.96, staircase=True)    
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

initial [16, 14, 14, 32] : [16, 28, 28, 32] :: [32]
second [16, 14, 14, 32] : [16, 14, 14, 32] :: [32]
third [16, 7, 7, 32] : [16, 14, 14, 32] :: [32]
initial [10000, 14, 14, 32] : [10000, 28, 28, 32] :: [32]
second [10000, 14, 14, 32] : [10000, 14, 14, 32] :: [32]
third [10000, 7, 7, 32] : [10000, 14, 14, 32] :: [32]
initial [10000, 14, 14, 32] : [10000, 28, 28, 32] :: [32]
second [10000, 14, 14, 32] : [10000, 14, 14, 32] :: [32]
third [10000, 7, 7, 32] : [10000, 14, 14, 32] :: [32]


In [104]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.053009
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.273938
Minibatch accuracy: 18.8%
Validation accuracy: 19.9%
Minibatch loss at step 100: 1.482131
Minibatch accuracy: 62.5%
Validation accuracy: 50.7%
Minibatch loss at step 150: 0.912264
Minibatch accuracy: 68.8%
Validation accuracy: 62.7%
Minibatch loss at step 200: 0.690222
Minibatch accuracy: 81.2%
Validation accuracy: 71.1%
Minibatch loss at step 250: 1.202375
Minibatch accuracy: 56.2%
Validation accuracy: 76.1%
Minibatch loss at step 300: 0.581289
Minibatch accuracy: 87.5%
Validation accuracy: 77.3%
Minibatch loss at step 350: 1.001937
Minibatch accuracy: 68.8%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.821719
Minibatch accuracy: 75.0%
Validation accuracy: 81.5%
Minibatch loss at step 450: 0.717112
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 500: 0.522096
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
M

Experiments done:
 - Added a 1X1 convolution
 - Max pooling layers
 - Tried avereage pooling -> this didn't really do much
 - Tried dropout -> this didn't really do much.
 - Tried learning rate decay -> this also didn't do much
 - Tried 1X1 convolution after the second dropout unit - this showed some improvement, but not much.
 
 However, what really works is:
 - Increasing the dept of the convolution
 - Increasing the number of steps.

Hence, we can infer that the network is not getting enough traning data.
Future work - Also try increasing the batch size, as we noticed that the minibatch accruacy is very high (indicating that the network is overfitting the mini-batches). 